# Imports

In [ ]:
from datetime import datetime
import pandas as pd
import math

# Reading events into DataFrames

In [ ]:
nb_lines = math.inf

clicks_content = []
i = 0
with open('lol', 'r') as f:
    for line in f:
        clicks_content.append(line.split(' ')[0])
        i += 1
        if i == nb_lines:
            break


displays_content = []
i = 0
with open('displays', 'r') as f:
    for line in f:
        displays_content.append(eval(line))
        i += 1
        if i == nb_lines:
            break

FileNotFoundError: ignored

In [ ]:
df_clicks = pd.DataFrame(clicks_content)
len(df_clicks.groupby(0).count())

50

In [ ]:
# time elapsed

datetime.fromtimestamp(df_clicks['timestamp'].max() - df_clicks['timestamp'].min())

datetime.datetime(1970, 1, 1, 0, 1, 6)

In [ ]:
# Too many IPs per user

a = df_clicks.groupby(['uid']).count()[['impressionId', 'ip']]
a[a['impressionId'] > 2]

,impressionId,ip
uid,,
0a76e964-d82b-4f00-b1b7-68996ce8a943,4,4
22010cc5-e086-4e4d-83c7-b2ae880a7b0b,3,3
27cd09ad-2360-4e73-9de7-dec4ecc8d6cc,4,4
2e7b2be5-284c-4e66-a1d6-cbd2706acb7c,5,5
3410d0a8-4521-4120-bd3a-f0a6e5849c96,6,6
37149369-50ad-46e1-ab77-fe846cdc386b,7,7
41aabe85-25de-4bf7-818a-47ee5b054427,3,3
4442cb69-15dc-4d48-90ee-f61238113dd7,3,3
463e0972-63b8-488b-a138-aa64d78e8678,4,4


In [ ]:
#convert timestamps to windows

window_size = 2 #seconds
df_clicks['time_window'] = df_clicks['timestamp'] // window_size

In [ ]:
df_windows = df_clicks.groupby(['uid', 'timewindow']).count()
df_windows[df_windows['impressionId'] > 2]

In [ ]:
df_clicks[df_clicks['ip'] == '238.186.83.58'].groupby('uid')

In [ ]:
print(len(df_clicks['impressionId']))
print(len(pd.unique(df_clicks['impressionId'])))

In [ ]:
df_displays = pd.DataFrame(displays_content)
df_displays

In [ ]:
# df_displays_renamed = df_displays.rename(columns={'uid' : 'uid_display', 'ip' : 'ip_display', 'impressionId' : 'impressionId_display', 'timestamp' : 'timestamp_display'})
df_displays_renamed = df_displays.rename(columns={'ip' : 'ip_display', 'timestamp' : 'timestamp_display'})
df_displays_renamed = df_displays_renamed.drop('eventType', axis=1)

In [ ]:
df_joined = pd.merge(df_displays_renamed, df_clicks, on=['uid', 'impressionId'])
df_joined

In [ ]:
ids_df_joined = df_joined[['uid', 'impressionId']]
ids_df_joined
i2 = ids_df_joined.set_index(['uid', 'impressionId']).index
i1 = df_clicks.set_index(['uid', 'impressionId']).index

In [ ]:
df_clicks[~i1.isin(i2)]

In [ ]:
a = df_clicks[df_clicks['uid'] == '57e9e910-07e8-4157-8120-59e269b7e503']
b = a[a['impressionId'] == 'ab2cc313-6fa5-42fe-9622-efd33214b5b9']
b

In [ ]:
df1 = pd.DataFrame({'c': ['A', 'A', 'B', 'C', 'C'],
                    'k': [1, 2, 2, 2, 2],
                    'l': ['a', 'b', 'a', 'a', 'd']})
df2 = pd.DataFrame({'c': ['A', 'C'],
                    'l': ['b', 'a']})
keys = list(df2.columns.values)
i1 = df1.set_index(keys).index
i2 = df2.set_index(keys).index
df1[~i1.isin(i2)]

In [ ]:
a = df_clicks.groupby(['ip']).count()
a[a['eventType'] > 3]

In [ ]:
print(len(df_clicks['impressionId'])/ len(df_displays['impressionId']))

In [ ]:
df

In [ ]:
df = pd.concat([df_displays, df_clicks])

topic_df = pd.get_dummies(df['eventType'])
df = df.drop('eventType', axis=1)
final_df = df.join(topic_df)
final_df = final_df.groupby('impressionId').count()
final_df['click_through_rate'] = final_df['click'] / final_df['display']
final_df

In [ ]:
df = pd.concat([df_displays, df_clicks])
df.groupby(['impressionId', 'eventType']).count()

In [ ]:
from datetime import datetime

datetime.fromtimestamp(1624772985)

In [ ]:
from collections import Set

l= []
with open('too_many_ips', 'r') as f:
    for line in f:
        l.append(line.split(',')[1])

In [ ]:
len(set(l))

50

In [9]:
lines = []
with open('CTR_UID_POST_PROCESSING', 'r') as f:
    for line in f:
        lines.append(float(line))

In [10]:
import math

print(sum(lines) / len(lines))

0.15373411976811657
